In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!pip install tweet-preprocessor
!pip install emoji --upgrade
import re
from emoji import demojize
import pandas as pd
import preprocessor as p
from collections import Counter
import nltk
nltk.download('wordnet')
import numpy as np
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.tokenize import TweetTokenizer
from nltk.tokenize.stanford import StanfordTokenizer 
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

  Created wheel for tweet-preprocessor: filename=tweet_preprocessor-0.5.0-cp36-none-any.whl size=7946 sha256=69988464e22a642a2c70ca00250998b83e1ba39c63e1908653dd616c5c2c8c5e
  Stored in directory: /root/.cache/pip/wheels/1b/27/cc/49938e98a2470802ebdefae9d2b3f524768e970c1ebbe2dc4a
Successfully built tweet-preprocessor
     |████████████████████████████████| 51kB 1.8MB/s 
  Created wheel for emoji: filename=emoji-0.5.3-cp36-none-any.whl size=42175 sha256=56635de7e7d7f95461aac0427cdef951802bf0344cabc7c04214c3935d92da53
  Stored in directory: /root/.cache/pip/wheels/86/09/26/f944015841423cd516e8a97f30e29be59e53461aea8b7d3458
Successfully built emoji
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [0]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import os
import pandas as pd
import re
import keras.layers as layers
from collections import Counter
from keras import backend as K
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.callbacks import TensorBoard
from keras.layers import Input, SpatialDropout1D, Embedding, BatchNormalization, LSTM, Dense, Concatenate
from keras.models import Model



Using TensorFlow backend.


In [0]:
!ls '/content/drive/My Drive/Final_Practicum/stance'

labelled_tweets.csv  stance_cleaned.csv      Stance_lstm.ipynb
model.h5	     Stance_Detection.ipynb  Stance_Prediction.csv


In [0]:
data = pd.read_csv('/content/drive/My Drive/Final_Practicum/stance/labelled_tweets.csv', encoding ='latin-1')
data = data[['full_text', 'Stance']]
data.head(10)

,full_text,Stance
0,is trending more than @INCIndia @BJP4India Hop...,BJP
1,RT @karna076: Do check out this rap!,BJP
2,PM Modi's Lok Sabha performance in three words...,BJP
3,'What use is a degree if I sell eggs?': The jo...,BJP
4,PM: India will be friends with both Israel &am...,BJP
5,RT @HimanshiKunwar1: Today modi ji is like @na...,BJP
6,RT @ManojGoelBJP: Watch @narendramodi's broadc...,BJP
7,RT @aditya_jachak: Today's Pic- 's Chief Whip ...,BJP
8,RT @keshaboinasri: The full majority will come...,BJP
9,"PM , while replying to the Motion of Thanks on...",BJP


In [0]:
def deEmojify(inputString):
    return inputString.encode('ascii', 'ignore').decode('ascii')
  
for index, row in data.iterrows():
  row['full_text']=deEmojify(row['full_text'])
  row['full_text'] = ' '.join(re.sub("(@[\w]*)|(#[A-Za-z0-9]+) | *RT*", " ", row['full_text']).split())
  row['full_text'] = ' '.join(re.sub("(\w+:\/\/\S+)", " ", row['full_text']).split())
  row['full_text'] = ' '.join(re.sub("[\.\,\!\?\:\;\-\=]", " ", row['full_text']).split())
  row['full_text'] = row['full_text'].replace("[^a-zA-Z ]", "")

data['full_text']=data['full_text'].astype(str)
data['full_text'] = data['full_text'].str.strip()
data[data.isnull().any(axis=1)]
data.fillna('', inplace=True)
data['full_text'].head(10)


0    is trending more than Hope I won't c ever Mamt...
1                                Do check out this rap
2    PM Modi's Lok Sabha performance in three words...
3    'What use is a degree if I sell eggs ' The job...
4    PM India will be friends with both Israel &amp...
5                                Today modi ji is like
6             Watch 's broadcast Speaking in the Watch
7    Today's Pic 's Chief Whip in Shri Ji Shri Ji &...
8    The full majority will come It also came in 20...
9    PM while replying to the Motion of Thanks on t...
Name: full_text, dtype: object

In [0]:
def in_dict(word):
    if wordnet.synsets(word):
        #if the word is in the dictionary, we'll return True
        return True

def replace_elongated_word(word):
    regex = r'(\w*)(\w+)\2(\w*)'
    repl = r'\1\2\3'    
    if in_dict(word):
        return word
    new_word = re.sub(regex, repl, word)
    if new_word != word:
        return replace_elongated_word(new_word)
    else:
        return new_word

def detect_elongated_words(row):
    regexrep = r'(\w*)(\w+)(\2)(\w*)'
    words = [''.join(i) for i in re.findall(regexrep, row)]
    for word in words:
        if not in_dict(word):
            row = re.sub(word, replace_elongated_word(word), row)
    return row

In [0]:
data['full_text'] = data['full_text'].apply(lambda x: detect_elongated_words(x))

In [0]:
lemmatizer = WordNetLemmatizer() 
def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

    return tag_dict.get(tag, wordnet.NOUN)

In [0]:
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [0]:
cleaned_tweets=[]
for index,row in data.iterrows():
    stopWords = set(stopwords.words('english'))
    tokens = nltk.word_tokenize(row["full_text"])
    rm_stopwords= [w for w in tokens if w not in stopWords]
    lemmatization=' '.join([lemmatizer.lemmatize(word , get_wordnet_pos(word)) for word in rm_stopwords if word])
    cleaned_tweets.append(lemmatization)
data['full_text']=pd.Series(cleaned_tweets)
data.head(10)

,full_text,Stance
0,trend Hope I wo n't c ever Mamtabanerje trend ...,BJP
1,Do check rap,BJP
2,PM Modi 's Lok Sabha performance three word I ...,BJP
3,'What use degree I sell egg ' The job crisis Modi,BJP
4,PM India friend Israel & amp Palestine India f...,BJP
5,Today modi ji like,BJP
6,Watch 's broadcast Speaking Watch,BJP
7,Today 's Pic 's Chief Whip Shri Ji Shri Ji & a...,BJP
8,The full majority come It also come 2014 also ...,BJP
9,PM reply Motion Thanks 's address dubbed oppos...,BJP


In [0]:
data.to_csv('/content/drive/My Drive/Final_Practicum/stance/stance_cleaned.csv', header=True, index=False)